In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/car-price-dataset/CarPriceDataset.csv


In [3]:
pd.__version__

'2.0.3'

In [4]:
file_path = '/kaggle/input/car-price-dataset/CarPriceDataset.csv'
df = pd.read_csv(file_path)

In [5]:
print(df.shape[1])

10


In [6]:
missing_columns = df.columns[df.isna().any()].tolist()
for c in missing_columns:
    print(c)

total_bedrooms


In [7]:
print(df['ocean_proximity'].nunique())

5


In [8]:
print(df[df['ocean_proximity'] == 'NEAR BAY']['median_house_value'].mean())

259212.31179039303


In [9]:
island_properties = df[df['ocean_proximity'] == 'ISLAND']
selected_columns = island_properties[['housing_median_age', 'total_rooms', 'total_bedrooms']]
X = selected_columns.to_numpy()
XTX = X.T.dot(X)
XTX_inverse = np.linalg.inv(XTX)
y = np.array([950, 1300, 800, 1000, 1300])
w = XTX_inverse.dot(X.T).dot(y)
last_element_of_w = w[-1]
print(last_element_of_w)

5.699229455065586


# Homework 2

In [10]:
filtered_df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]
selected_columns = ['latitude', 'longitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value']
subset_df = filtered_df[selected_columns]
missing_values = subset_df.isnull().sum()
print(missing_values)

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        157
population              0
households              0
median_income           0
median_house_value      0
dtype: int64


In [12]:
print(filtered_df['population'].median())

1195.0


In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

np.random.seed(42)
encoded_df = pd.get_dummies(filtered_df, columns=['ocean_proximity'])
shuffled_df = encoded_df.sample(frac=1, random_state=42)

train_size = 0.6
val_size = 0.2
test_size = 0.2

train_df, val_test_df = train_test_split(shuffled_df, test_size=val_size + test_size, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=test_size/(val_size + test_size), random_state=42)

train_df['median_house_value'] = np.log1p(train_df['median_house_value'])
val_df['median_house_value'] = np.log1p(val_df['median_house_value'])
test_df['median_house_value'] = np.log1p(test_df['median_house_value'])

median_bedrooms = train_df['total_bedrooms'].mean()
train_df['total_bedrooms'].fillna(median_bedrooms, inplace=True)
val_df['total_bedrooms'].fillna(median_bedrooms, inplace=True)

X_train = train_df.drop(columns=['median_house_value'])
y_train = train_df['median_house_value']
X_val = val_df.drop(columns=['median_house_value'])
y_val = val_df['median_house_value']

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("RMSE:", round(rmse, 2))

RMSE: 0.33


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

np.random.seed(42)
encoded_df = pd.get_dummies(filtered_df, columns=['ocean_proximity'])
shuffled_df = encoded_df.sample(frac=1, random_state=42)

train_size = 0.6
val_size = 0.2
test_size = 0.2

train_df, val_test_df = train_test_split(shuffled_df, test_size=val_size + test_size, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=test_size/(val_size + test_size), random_state=42)

train_df['median_house_value'] = np.log1p(train_df['median_house_value'])
val_df['median_house_value'] = np.log1p(val_df['median_house_value'])
test_df['median_house_value'] = np.log1p(test_df['median_house_value'])

train_df.fillna(0, inplace=True)
val_df.fillna(0, inplace=True)

X_train = train_df.drop(columns=['median_house_value'])
y_train = train_df['median_house_value']
X_val = val_df.drop(columns=['median_house_value'])
y_val = val_df['median_house_value']

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print("RMSE:", round(rmse, 2))

RMSE: 0.33


In [20]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from math import sqrt

alphas = [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]
rmse_scores = []
models = []

train_df, val_test_df = train_test_split(shuffled_df, test_size=val_size + test_size, random_state=42)
val_df, test_df = train_test_split(val_test_df, test_size=test_size/(val_size + test_size), random_state=42)

train_df['median_house_value'] = np.log1p(train_df['median_house_value'])
val_df['median_house_value'] = np.log1p(val_df['median_house_value'])
test_df['median_house_value'] = np.log1p(test_df['median_house_value'])

train_df.fillna(0, inplace=True)
val_df.fillna(0, inplace=True)

X_train = train_df.drop(columns=['median_house_value'])
y_train = train_df['median_house_value']
X_val = val_df.drop(columns=['median_house_value'])
y_val = val_df['median_house_value']

for alpha in alphas:
    model = Ridge(alpha=alpha)
    model.fit(X_train, y_train)
    models.append(model)
    y_pred = model.predict(X_val)
    rmse = sqrt(mean_squared_error(y_val, y_pred))
    rmse_scores.append(rmse)
    
best_alpha_index = np.argmin(rmse_scores)
best_alpha = alphas[best_alpha_index]
best_rmse = rmse_scores[best_alpha_index]
print(f'Best alpha: {best_alpha}, RMSE: {round(best_rmse, 2)}')

Best alpha: 0, RMSE: 0.33


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=2.48878e-24): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=1.09458e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T


In [40]:

from sklearn.metrics import mean_squared_error

seed_values = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
rmse_scores = []

encoded_df.fillna(0, inplace=True)
shuffled_df = encoded_df.sample(frac=1, random_state=42)

X = shuffled_df.drop(columns=['median_house_value'])
y = shuffled_df['median_house_value']

for seed in seed_values:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    rmse = sqrt(mean_squared_error(y_train, y_pred))
    rmse_scores.append(rmse)

std_deviation = np.std(rmse_scores)
print(round(std_deviation, 3))
print (rmse_scores)

272.161
[62899.70421915131, 63390.14553398527, 63155.8500420849, 63830.66234501531, 63771.12745323719, 63644.791763116824, 63621.64569918421, 63537.69225993293, 63425.817355623665, 63657.30223300517]


In [41]:
seed = 9

encoded_df.fillna(0, inplace=True)
shuffled_df = encoded_df.sample(frac=1, random_state=42)

train_size = 0.6
val_size = 0.2
test_size = 0.2

X_train_val = shuffled_df.drop(columns=['median_house_value'])
y_train_val = shuffled_df['median_house_value']
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, random_state=seed)

model = Ridge(alpha=0.001)
model.fit(X_train_val, y_train_val)

y_pred = model.predict(X_test)
rmse_test = sqrt(mean_squared_error(y_test, y_pred))
print("RMSE on the test dataset:", rmse_test)

RMSE on the test dataset: 66567.12944256622
